In [ ]:
! pip install -r requirements.txt --quiet

# Connecting to a Remote MCP Server with LangChain

This notebook demonstrates how to connect to a remote **MCP Server** using LangChain's **MultiServerMCPClient**.

- Use the **Model Context Protocol (MCP)** to modularize and scale tool usage across distributed systems.
- Define and call tools hosted remotely, enabling seamless agent interaction and task execution.
- Showcase how MCP enables flexible, service-oriented tool integration for multi-agent workflows.

<br/>

> **Why Use Model Context Protocol (MCP)?**
>
>MCP allows agents to discover, invoke, and manage tools dynamically across remote servers.  
>It promotes modularity, scalability, and separation of concerns, making it easier to maintain and extend AI systems as they grow in complexity.


In [ ]:
from dotenv import load_dotenv
from os import environ
from langchain_openai import AzureChatOpenAI
from utils import pretty_print_response
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.messages import SystemMessage, HumanMessage
from langchain.agents import create_agent
from azure.identity import DefaultAzureCredential, get_bearer_token_provider


load_dotenv(override=True)

In [ ]:
# Use Azure default credentials instead of API key
token_provider = get_bearer_token_provider(
    DefaultAzureCredential(),
    "https://cognitiveservices.azure.com/.default"
)

#environ["AZURE_OPENAI_API_VERSION"] = token_provider.

In [ ]:
llm = AzureChatOpenAI(
        temperature=0,
        azure_deployment=environ["MODEL_DEPLOYMENT_NAME"],
        api_version=environ["AZURE_OPENAI_API_VERSION"],
        azure_ad_token_provider=token_provider
    )

## Connecting to a remote MCP server via sse

- Server-Sent Events (SSE) is a mechanism for establishing a continuous, one-way data stream from the server to a client, such as a web application. This allows the server to push updates or messages to the client without the client needing to constantly poll for new information

In [10]:

client = MultiServerMCPClient( {
'mcp-contract' : {
        "url": 'http://135.18.211.181/mcp',
        "transport": "http"
    },
    'mcp-market' : {
        "url": 'http://135.222.157.234/mcp',
        "transport": "http"
    },
    'mcp-risk' : {
        "url": 'http://135.224.254.1/mcp',
        "transport": "http"
    }
}
)

tools = await client.get_tools()  


agent = create_agent(
    llm,
    tools=tools  
)


reponse = await agent.ainvoke({"messages": "Analyze this threshold breach and provide recommendations"})
pretty_print_response(reponse)






Human: Analyze this threshold breach and provide recommendations


AI: To analyze the threshold breach and provide recommendations, I need more details about the breach. Specifically:

1. **Contract Details**: Which contract is associated with the breach? Provide the contract ID or relevant details.
2. **Threshold Breach Information**: What threshold was breached? For example, was it related to VaR, DV01, FX rate, volatility, or another metric?
3. **Risk Metrics**: If available, share the risk metrics or data associated with the breach.
4. **Context**: Any additional context or background information about the breach.

Once I have this information, I can analyze the situation and provide recommendations.


